In [ ]:
import tensorflow as tf
import numpy as np
import collections
import math

### Create class for model:
***

In [ ]:
class Model:
    def __init__(self, nStates, nActions, nBatches):
        
        ### Misc variables ###
        self._nStates = nStates
        self._nBatches = nBatches
        self._nActions = nActions
        
        ### Placeholders ###
        self._states = None
        self._actions = None
        
        ### outputs ###
        self._logits = None
        self._optimizer = None
        self._var_init = None
        
        self._define_model()
        
    def _define_model(self):
        self._states = tf.placeholder(shape=[None, self._nStates],
                                      dtype=tf.float32)
        self._qsa = tf.placeholder(shape=[None, self._nActions],
                                   dtype = tf.float32)
        
        ### layers ###
        x1 = tf.layers.dense(self._states, 50, activation=tf.nn.relu)
        x2 = tf.layers.dense(x1, 50, activation=tf.nn.relu)
        
        ### outputs ###
        self._logits = tf.layers.dense(x2, self._nActions)
        
        ### loss ###
        loss = tf.losses.mean_squared_error(self._qsa, self._logits)
        self._optimizer = tf.train.AdamOptimizer().minimize(loss)
        self._var_init = tf.global_variables_initializer()
        
    def predict_one(self, state, sess):
        dict2feed = {self._states : state.reshape(1, self._nStates)}
        return sess.run(self._logits, feed_dict=dict2feed)
    
    def predict_batch(self, state, sess):
        dict2feed = {self._states : state}
        return sess.run(self._logits, feed_dict = dict2feed)
    
    def train_batch(self, sess, x_bach, y_batch):
        dict2feed = {self._states: x_bach,
                     self._qsa : y_batch}
        sess.run(self._optimizer, feed_dict = dict2feed)
        

### Create class to store (state, action) pairs
***

In [ ]:
class Memory:
    def __init__(self, maxMemory):
        self._samples = collections.deque(maxlen = maxMemory)
        
    def add_sample(self, sample):
        self._samples.append(sample)
        
    # def sample(self, nSamples):
    #     deqLen = len(self._samples)
    #     if nSamples > deqLen: return np.random.sample(self._samples, deqLen)
    #     else: return np.random.sample(self._samples, nSamples)
        
    def sample(self, nSamples):
        deqLen = len(self._samples)
        high = np.max([deqLen-1, 1])
        
        ### DEBUG ###
        #print ("dequelen: {}".format(deqLen))
        #print ("nSamples: {}".format(nSamples))
        #print ("High: {}".format(high))
        
        size = nSamples if nSamples <= deqLen else deqLen
        
        randInts = np.random.randint(low = 0,
                                     high = high,
                                     size = size)
        return [self._samples[i] for i in randInts]
        

### GameRunner class:
***

In [ ]:
class GameRunner:
    def __init__(self, sess, model, env, memory,
                 max_eps, min_eps, gamma, render=True):
        self._sess = sess
        self._env = env
        self._model = model
        self._memory = memory
        self._render = render
        self._max_eps = max_eps
        self._min_eps = min_eps
        self._decay = gamma
        self._eps = self._max_eps
        self._steps = 0
        self._reward_store = []
        self._max_x_store = []
        
        self._gamma = 0.5
    
    def run(self):
        
        state = self._env.reset()
        tot_reward = 0
        max_x = -100
        
        while True:
            if self._render: self._env.render()    # If render is TRUE, render the environment to screen 
            action = self._choose_action(state)    # choose an action based on the state
            next_state, reward, done, info = self._env.step(action)    # pick action, get reward and new state
            
            ### Custom increase to reward function ###
            if next_state[0] >= 0.1: reward += 10
            elif next_state[0] >= 0.25: reward += 20
            elif next_state[0] >= 0.5: reward += 100
            if next_state[0] > max_x: max_x = next_state[0]    # increase max_x if appropriate
            
            ### Is game complete? ###
            if done: next_state = None
            
            ### Memory ###
            self._memory.add_sample((state, action, reward, next_state))
            self._replay()
            
            ## Exponentially decay the epsilon value
            self._steps += 1
            self._eps = self._min_eps + (self._max_eps - self._min_eps) \
                                  * math.exp(-self._decay * self._steps)
            
            ### move the agent to the next state and accumulate the reward ###
            state = next_state
            tot_reward += reward
            
            ### if game is done, break loop ###
            if done:
                self._reward_store.append(tot_reward)
                self._max_x_store.append(max_x)
                break
        
        print ("Step {}, Total reward: {}, Eps: {}".format(self._steps, tot_reward, self._eps))
        
        
    def _choose_action(self, state):
        if np.random.random() < self._eps: return np.random.randint(0, self._model._nActions - 1)
        else: return np.argmax(self._model.predict_one(state, self._sess))
        
    def _replay(self):
        batch = self._memory.sample(self._model._nBatches)
        states = np.array([ x[0] for x in batch ])
        next_states = np.array([ np.zeros(self._model._nStates) if x[3] is None else x[3] for x in batch])
        
        qsa = self._model.predict_batch(states, self._sess)
        qsad = self._model.predict_batch(next_states, self._sess)
        
        x = np.zeros((len(batch), self._model._nStates))
        y = np.zeros((len(batch), self._model._nActions))
        
        for i, b in enumerate(batch):
            
            state, action, reward, next_state = b[0], b[1], b[2], b[3]
            
            current_q = qsa[i]
            
            if next_state is None: current_q[action] = reward
            else: current_q[action] = reward + self._gamma * np.amax(qsad[i])
                
            x[i] = state
            y[i] = current_q
            
        self._model.train_batch(self._sess, x, y)
            
            

### Main function
***

In [ ]:
import gym
import matplotlib.pyplot as plt

In [ ]:
BATCH_SIZE = 32
MAX_EPSILON = 0.5
MIN_EPSILON = 0.0
GAMMA = 0.5

In [ ]:
env_name = 'MountainCar-v0'
env = gym.make(env_name)

In [ ]:
nStates = env.env.observation_space.shape[0]
nActions = env.env.action_space.n

In [ ]:
model = Model(nStates, nActions, BATCH_SIZE)

In [ ]:
mem = Memory(10000)

In [ ]:
with tf.Session() as sess:
    sess.run(model._var_init)
    gr = GameRunner(sess, model, env, mem, MAX_EPSILON, MIN_EPSILON, GAMMA, render=False)
    num_episodes = 300
    count = 0
    while count < num_episodes:
        if count % 10 == 0:
            print ('Episode {} of {}'.format(count+1, num_episodes))
        gr.run()
        count += 1
    
    plt.plot(gr.reward_store)
    plt.show()
    plt.close("all")
    plt.plot(gr.max_x_store)
    plt.show